In [6]:
import os
import scipy.io
import numpy as np
import cv2
from tqdm import tqdm # Thanh hiển thị tiến độ

# --- CẤU HÌNH ---
INPUT_FOLDER = r"D:\project\2015_BOE_Chiu"  # Thư mục chứa file .mat
OUTPUT_FOLDER = r"D:\project\processed_data" # Nơi lưu ảnh sau khi xử lý

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def normalize_image(img):
    # Chuẩn hóa ảnh về 0-255 để lưu file PNG
    img = img.astype(float)
    img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8) # Tránh chia cho 0
    img = (img * 255).astype(np.uint8)
    return img

def process_mat_files():
    # Tạo thư mục đích
    img_dir = os.path.join(OUTPUT_FOLDER, "images")
    mask_dir = os.path.join(OUTPUT_FOLDER, "masks")
    create_dir(img_dir)
    create_dir(mask_dir)
    
    # Lấy danh sách file .mat
    files = [f for f in os.listdir(INPUT_FOLDER) if f.endswith('.mat')]
    
    print(f"Tìm thấy {len(files)} bệnh nhân. Bắt đầu xử lý...")
    
    for file_name in tqdm(files):
        file_path = os.path.join(INPUT_FOLDER, file_name)
        subject_id = file_name.split('.')[0] # Lấy tên "Subject_01"
        
        try:
            mat = scipy.io.loadmat(file_path)
            
            # Kiểm tra key
            if 'images' not in mat or 'manualFluid1' not in mat:
                print(f"Bỏ qua {file_name}: Thiếu key images hoặc manualFluid1")
                continue
                
            images = mat['images']       # (H, W, Slices)
            masks = mat['manualFluid1']  # (H, W, Slices)
            
            num_slices = images.shape[2]
            
            for i in range(num_slices):
                # Lấy từng lát cắt
                img_slice = images[:, :, i]
                mask_slice = masks[:, :, i]
                
                # Xử lý mask: Chuyển NaN thành 0, lớn hơn 0 thành 1 (hoặc 255)
                mask_slice = np.nan_to_num(mask_slice)
                mask_slice = (mask_slice > 0).astype(np.uint8) * 255
                
                # Bỏ qua các lát cắt quá nhiễu hoặc lỗi (nếu cần - hiện tại giữ hết)
                
                # Chuẩn hóa ảnh gốc
                img_slice = normalize_image(img_slice)
                
                # Lưu file
                save_name = f"{subject_id}_slice_{i:03d}.png"
                
                cv2.imwrite(os.path.join(img_dir, save_name), img_slice)
                cv2.imwrite(os.path.join(mask_dir, save_name), mask_slice)
                
        except Exception as e:
            print(f"Lỗi khi xử lý {file_name}: {e}")

    print("\n--- Xử lý hoàn tất! ---")
    print(f"Dữ liệu đã lưu tại: {OUTPUT_FOLDER}")

# Chạy hàm
if __name__ == "__main__":
    process_mat_files()

Tìm thấy 10 bệnh nhân. Bắt đầu xử lý...


100%|██████████| 10/10 [00:37<00:00,  3.80s/it]


--- Xử lý hoàn tất! ---
Dữ liệu đã lưu tại: D:\project\processed_data
